In [18]:
from alive_progress import alive_bar
from IPython.display import clear_output
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import regex as re

In [19]:
# Definitions
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
PARENT_DIR = os.path.dirname(ROOT_DIR)

def replaceAll(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [24]:
urls = []

num_pages = 880 # Number of pages per country (rus,ukr)

with alive_bar(2*num_pages, title="-> Scraper", spinner="dots_waves", bar="smooth", force_tty=True) as bar:
    for page in range (1,2*(num_pages)+1):

        ukr = "https://www.reuters.com/news/archive/ukraine?view=page&page=" + str(page) + "&pageSize=10"
        rus = "https://www.reuters.com/news/archive/russia?view=page&page=" + str(page) + "&pageSize=10"
        tags = [ukr,rus]


        for tag in tags:
            html_text = requests.get(tag).text
            soup = BeautifulSoup(html_text, 'lxml')
            headline_list = soup.find('div', class_='column1 col col-10')
            headlines = headline_list.find_all('div', class_='story-content')

            for headline in headlines:
                page_urls = headline.find_all('a', href=True)
                for _ in page_urls:
                    urls.append("https://www.reuters.com" + _['href'])
            
        bar()

unique_urls = list(dict.fromkeys(urls))
print(f"-> {len(unique_urls)} URLs fetched successfully!")   


-> Scraper |████████████████████████████████████████| 2/2 [100%] in 0.9s (2.18/s)                                       
-> 29 URLs fetched successfully!


In [21]:
bodies = []
titles = []
dates = []
urls = []

# Dict of undesirable substrings
rep = {
    "Our Standards: The Thomson Reuters Trust Principles.": "",
    "read more": "",
}

with alive_bar(len(unique_urls), title="-> Scraper", spinner="dots_waves", bar="smooth", force_tty=True) as bar:
    for url in unique_urls:
        try:

            title_tags = ["text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_2__1K_hh heading__base__2T28j heading__heading_2__3Fcw5", "Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj"] 
            date_tags = ["date-line__date__23Ge-", "ArticleHeader-date-line-3oc3Y"]
            text_tags = ["text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__large__nEccO body__base__22dCE body__large_body__FV5_X article-body__element__2p5pI"]

            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, "lxml")

            title = soup.find("h1", class_= title_tags).text
            date = soup.find("span", class_= date_tags).text
            paragraphs = soup.find_all("p", class_= text_tags)
            body = ""
            for _ in paragraphs:
                body += " " + _.text
            
            body = replaceAll(body, rep)
            bodies.append(re.sub(r"^[^-]*-", "", " ".join(body.split())))

            titles.append(title)
            dates.append(date)
            urls.append(url)
            bar()
        except:
            print(f"URL could'nt be parsed:{url}")
            pass

data = pd.DataFrame({"URL": urls, "Date": dates, "Title": titles, "Text": bodies})
os.makedirs(PARENT_DIR + "/data", exist_ok=True)
data.to_csv(PARENT_DIR + "/data/Reuters.csv", index=True, header=True)

print("-> Database generated successfully!")


-> Scraper |████████████████████████████████████████| 29/29 [100%] in 30.4s (0.95/s)                                    
Database generated successfully!


In [25]:
data.head()

,URL,Date,Title,Text
0,https://www.reuters.com/article/us-usa-saudi-a...,"July 11, 2022",EXCLUSIVE U.S. weighs resumption of offensive ...,The Biden administration is discussing the po...
1,https://www.reuters.com/article/us-ukraine-cri...,"July 11, 2022",Ukraine and Russia: What you need to know righ...,Rescue workers pulled more bodies and some su...
2,https://www.reuters.com/article/us-ukraine-cri...,"July 11, 2022",Rescuers pull survivors from ruined Ukrainian ...,Rescuers pulled survivors on Monday from an a...
3,https://www.reuters.com/article/syria-crisis-u...,"July 11, 2022",U.N. aid to Syria from Turkey likely to contin...,The U.N. Security Council appears set to allo...
4,https://www.reuters.com/article/us-britain-pol...,"July 11, 2022",New UK prime minister to be announced on Sept....,Britain's new prime minister will be announce...
